<a href="https://colab.research.google.com/github/cswcjt/Quant-Project/blob/main/frame_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance --quiet

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize

In [3]:
def get_price(tickers: list, period: str, interval: str, start_date: str=None) -> pd.DataFrame:
    '''
    기능: interval 가격 정보를 받는다.

    period, interval: 분 단위 정보 -> 회정 자산배분을 위해 필요
    start_date: 
    interval 변수: (1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo)
    '''
    
    temp = []
    for ticker in tickers:
        name = f'{ticker}'
        name = yf.Ticker(ticker)
        temp_df = name.history(start=start_date, period=period, interval=interval)['Close']
        temp.append(temp_df)
    
    price_df = pd.concat(temp, axis = 1)
    price_df.columns = tickers
    
    if interval in ['1d', '5d', '1wk', '1mo', '3mo']:
        price_df.index = pd.to_datetime(price_df.index.date)
    
    price_df.dropna(inplace=True)

    return price_df

In [4]:
risky_asset = ['GOOGL', 'KO', 'HD', 'INTU', 'PEP', 'NOW', 'TSLA', 'UNH']
bond_asset = ["SHV", "IEF", "TLT", 'IAU', 'SLV', 'VNQ']
canary_asset = ["VWO", "BND"]
#econ_ind = ['UNRATE', 'SP500']
all_tickers = risky_asset + bond_asset + canary_asset

price_df = get_price(all_tickers, 'max', '1d')
price_df

,GOOGL,KO,HD,INTU,PEP,NOW,TSLA,UNH,SHV,IEF,TLT,IAU,SLV,VNQ,VWO,BND
2012-06-29,14.516266,28.154282,41.959534,53.893162,52.216064,24.600000,2.086000,49.762318,102.687897,89.848320,97.563148,31.120001,26.650000,43.129219,29.418522,64.217361
2012-07-02,14.526276,28.417143,41.975361,54.192833,52.289963,24.770000,2.026667,47.856895,102.678558,90.265190,98.569763,31.120001,26.730000,43.603813,29.447989,64.352333
2012-07-03,14.710460,28.503561,40.898468,54.619621,52.289963,25.309999,2.044000,46.683018,102.687897,90.040062,97.812233,31.600000,27.490000,43.860889,29.985821,64.260841
2012-07-05,14.912913,28.247898,41.262711,54.410770,51.853973,25.959999,2.082000,47.474098,102.678558,90.281837,98.312035,31.260000,26.879999,43.590626,29.727955,64.443848
2012-07-06,14.664164,28.139881,41.294384,52.708431,51.890926,25.840000,2.066000,47.482609,102.687897,90.598656,99.225731,30.840000,26.299999,43.696091,29.190125,64.504913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30,88.230003,63.610001,315.859985,389.220001,180.660004,388.269989,123.180000,530.179993,109.919998,95.779999,99.559998,34.590000,22.020000,82.480003,38.980000,71.839996
2023-01-03,89.120003,62.950001,315.910004,391.179993,179.410004,385.500000,108.099998,518.640015,109.940002,96.529999,101.459999,34.880001,22.049999,82.559998,39.400002,72.220001
2023-01-04,88.080002,62.919998,319.730011,391.570007,178.970001,393.850006,113.639999,504.500000,109.949997,97.269997,102.849998,35.209999,21.889999,84.430000,40.419998,72.629997
2023-01-05,86.199997,62.200001,315.470001,375.619995,177.100006,366.320007,110.339996,489.959991,109.970001,97.129997,103.279999,34.790001,21.389999,82.160004,40.299999,72.550003


In [5]:
def add_cash(price: pd.DataFrame, num_day_in_year:int, yearly_rfr: int) -> pd.DataFrame:
    '''
    기능: 전체 자산의 가격정보 df에 현금을 복리로 계산한 col 추가한다.

    price: get_price()의 return 값
    num_day_in_year: 1년 중 실제 비즈니스 일 수(임의의 상수값): 252
    yearly_rfr: 무위험 수익률(임의의 상수값): 0.04
    '''
    
    temp_df = price.copy()

    temp_df['CASH'] = yearly_rfr/num_day_in_year
    temp_df['CASH'] = (1 + temp_df['CASH']).cumprod()
    temp_df.dropna(inplace = True)
    temp_df.index.name = "date_time"

    return temp_df

In [6]:
price_df = add_cash(price_df, 252, 0.04)
price_df.index

DatetimeIndex(['2012-06-29', '2012-07-02', '2012-07-03', '2012-07-05',
               '2012-07-06', '2012-07-09', '2012-07-10', '2012-07-11',
               '2012-07-12', '2012-07-13',
               ...
               '2022-12-22', '2022-12-23', '2022-12-27', '2022-12-28',
               '2022-12-29', '2022-12-30', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06'],
              dtype='datetime64[ns]', name='date_time', length=2648, freq=None)

In [7]:
def rebal_dates(price: pd.DataFrame, period: str) -> list: 
    '''
    기능: 포트폴리오 리밸런싱 날을 구한다.

    price: get_price()의 결과값
    period: 리밸런싱의 주기를 설정
    '''
    
    _price = price.reset_index()
    
    if period == "month":
        groupby = [_price['date_time'].dt.year, _price['date_time'].dt.month]
         
    elif period == "quarter":
        groupby = [_price['date_time'].dt.year, _price['date_time'].dt.quarter]
        
    elif period == "halfyear":
        groupby = [_price['date_time'].dt.year, _price['date_time'].dt.month // 7]
        
    elif period == "year":
        groupby = [_price['date_time'].dt.year, _price['date_time'].dt.year]
        
    rebal_dates = pd.to_datetime(_price.groupby(groupby)['date_time'].last().values)
    
    return rebal_dates


In [8]:
rebal_dates = rebal_dates(price_df, 'month')
rebal_dates

DatetimeIndex(['2012-06-29', '2012-07-31', '2012-08-31', '2012-09-28',
               '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31',
               '2013-02-28', '2013-03-28',
               ...
               '2022-04-29', '2022-05-31', '2022-06-30', '2022-07-29',
               '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
               '2022-12-30', '2023-01-06'],
              dtype='datetime64[ns]', length=128, freq=None)

In [9]:
def price_on_rebal(price: pd.DataFrame, rebal_dates: list) -> pd.DataFrame:
    '''
    기능: 리밸런싱 날의 가격을 갖고 있는 df

    price: get_price()의 결과값
    rebal_dates: rebal_dates의 결과값
    '''

    price_on_rebal = price.loc[rebal_dates, :]
    return price_on_rebal

In [10]:
price_on_rebal_df = price_on_rebal(price_df, rebal_dates)
price_on_rebal_df

,GOOGL,KO,HD,INTU,PEP,NOW,TSLA,UNH,SHV,IEF,TLT,IAU,SLV,VNQ,VWO,BND,CASH
2012-06-29,14.516266,28.154282,41.959534,53.893162,52.216064,24.600000,2.086000,49.762318,102.687897,89.848320,97.563148,31.120001,26.650000,43.129219,29.418522,64.217361,1.000159
2012-07-31,15.840090,29.094076,41.318142,52.817684,53.745754,27.000000,1.828000,43.459087,102.687897,91.107201,101.287422,31.440001,27.120001,43.992714,29.477461,65.008339,1.003498
2012-08-31,17.144394,26.933626,45.167988,53.291050,53.524067,31.100000,1.901333,46.189632,102.706535,91.000366,99.951515,32.959999,30.790001,43.986126,29.551140,65.112984,1.007168
2012-09-28,18.881380,27.500971,48.049187,53.600567,52.688442,38.680000,1.952000,47.324772,102.698135,90.683449,97.419945,34.540001,33.480000,43.166584,31.120388,65.231003,1.010210
2012-10-31,17.024525,26.957193,48.853058,54.252274,51.549355,30.650000,1.875333,47.828671,102.690674,90.324280,96.948425,33.520000,31.270000,42.774578,30.948816,65.165001,1.013582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,95.650002,55.625496,274.280670,386.589539,162.247925,377.609985,265.250000,503.487946,108.957878,95.221710,101.467422,31.530001,17.500000,79.052689,35.903538,70.680969,1.506291
2022-10-31,94.510002,59.428520,294.349274,427.500000,180.454361,420.739990,227.539993,553.443970,109.127579,93.837166,95.415474,30.980000,17.620001,81.823532,34.880253,69.862671,1.511319
2022-11-30,100.989998,63.610001,323.989990,407.589996,184.360001,416.299988,194.699997,546.076660,109.497536,97.226807,102.238693,33.599998,20.430000,86.852455,39.868771,72.423790,1.516365
2022-12-30,88.230003,63.610001,315.859985,389.220001,180.660004,388.269989,123.180000,530.179993,109.919998,95.779999,99.559998,34.590000,22.020000,82.480003,38.980000,71.839996,1.521428


In [11]:
# 주식 모멘텀 클래스
class Momentum:
    '''
    기능: 모멘텀 전략들을 클래스로 관리한다.
    '''

    # 초기화 함수
    def __init__(self, price: pd.DataFrame, 
                 lookback_window: int, n_sel: int, 
                 long_only: bool=True):
        '''
        price: 리밸런싱 날짜의 자산가격 df -> price_on_rebal()의 결과값
        lookback_window: 추세선의 기준
        n_sel: 시그널이 있는 자산들 중 몇 개를 고를지 정한다
        long_only: 매수, 공매도 가능여부 설정
        '''

        self.price = price
        self.lookback_window = lookback_window
        self.rets = price.pct_change(self.lookback_window).fillna(0)
        self.n_sel = n_sel

    # 절대 모멘텀 시그널 계산 함수
    def absolute_momentum(self, long_only: bool=True):
 
        returns = self.rets

        # 롱 시그널
        long_signal = (returns > 0) * 1

        # 숏 시그널
        short_signal = (returns < 0) * -1

        # 토탈 시그널
        if long_only:
            signal = long_signal

        else:
            signal = long_signal + short_signal
        
        return signal
    
    # 상대 모멘텀 시그널 계산 함수
    def relative_momentum(self, long_only: bool=True):

        # 수익률
        returns = self.rets

        # 자산 개수 설정
        n_sel = self.n_sel

        # 수익률 순위화
        rank = returns.rank(axis=1, ascending=False)

        # 롱 시그널
        long_signal = (rank <= n_sel) * 1

        # 숏 시그널
        short_signal = (rank >= len(rank.columns) - n_sel + 1) * -1

        # 토탈 시그널
        if long_only:
            signal = long_signal

        else:
            signal = long_signal + short_signal

        return signal
    
    # 듀얼 모멘텀 시그널 계산 함수
    def dual_momentum(self, long_only: bool=True):

        # 절대 모멘텀 시그널
        abs_signal = self.absolute_momentum(long_only)

        # 상대 모멘텀 시그널
        rel_signal = self.relative_momentum(long_only)

        # 듀얼 모멘텀 시그널
        signal = (abs_signal == rel_signal) * abs_signal

        return signal

    # def momentum_score(self, long_only: bool=True):
        
    #     price = self.price
    #     score_result = (12*price.pct_change(1)) + (4*price.pct_change(3)) \
    #                                                   + (2*price.pct_change(6)) \
    #                                                   + (1*price.pct_change(12))

    #     score_result = score_result.dropna()

    #     # 2.2 : MOMENTUM SCORE OF CANARY ASSET
    #     canary_signal_series = (score_result[canary_asset] > 0).sum(axis=1)
    #     canary_signal_series
    #     score_result['canary_signal'] = canary_signal_series
    #     score_result

    #     return signal


In [12]:
momfactor = Momentum(price_on_rebal_df, 1, 3)

In [13]:
signal = momfactor.dual_momentum()

In [14]:
signal.iloc[1].index

Index(['GOOGL', 'KO', 'HD', 'INTU', 'PEP', 'NOW', 'TSLA', 'UNH', 'SHV', 'IEF',
       'TLT', 'IAU', 'SLV', 'VNQ', 'VWO', 'BND', 'CASH'],
      dtype='object')

In [15]:
signal.iloc[1] != 0

GOOGL     True
KO       False
HD       False
INTU     False
PEP      False
NOW       True
TSLA     False
UNH      False
SHV      False
IEF      False
TLT       True
IAU      False
SLV      False
VNQ      False
VWO      False
BND      False
CASH     False
Name: 2012-07-31 00:00:00, dtype: bool

In [16]:
# 원자재 밸류 팩터 전략 구현 클래스
class Value:
    '''
    기능: 모멘텀 전략들을 클래스로 관리한다.
    '''
    
    # 초기화 함수
    def __init__(self, price):
        pass
        



In [17]:
# 변동성 캐리 전략 구현 클래스
class VolatilityCarry:
    '''
    기능: 모멘텀 전략들을 클래스로 관리한다.
    '''

    # 초기화 함수
    def __init__(self, price, slope):

        # 변동성 선물 가격
        self.price = price

        # 수익률
        self.rets = self.price.pct_change()

        # 기간구조 기울기
        self.slope = slope

        # 거래비용
        self.cost = 0.002

        # 가중치
        self.weights = self.calculate_weights(self.slope)

        # 포트폴리오 수익률
        self.port_rets = self.calculate_returns(self.rets, self.weights, self.cost)

        # 백테스팅 결과 시각화
        self.plot_port_returns(self.port_rets)

        # 샤프비율
        self.sharpe_ratio = self.calculate_sharpe_ratio(self.port_rets)

    # 가중치 계산 함수
    def calculate_weights(self, slope):
        
        # 롱 포지션
        long_weights = (slope < 1) * 1

        # 숏 포지션
        short_weights = (slope > 1) * -1

        # 토탈 포지션
        total_weights = long_weights + short_weights

        return total_weights



In [18]:
# 각 팩터의 시그널을 종합하는 단계

In [ ]:
# 종적 배분 모델 클래스 
# -> 팩터로 선별된 자산의 티커를 가지고 온다
# -> 연율화 시켜준다
# 근데 이때 샘플 크기를 늘리면 정규분포가 될까? -> anormally detection 적용 가능할까????
class CrossSectional:
    '''
    횡적 자산배분을 위해 포트폴리오 이론 적용
    '''

    # 초기화 함수
    def __init__(self, price: pd.DataFrame, signal: pd.DataFrame, param: int):
        '''
        price: 리밸런싱 날짜의 자산가격 df -> price_on_rebal()의 결과값
        signal: 팩터의 시그널을 담고 있는 df 
        param: 연율화를 위한 파라미터 -> ex)리밸런싱 기간 == 1달, parma = 12 
        '''
        # 거래대상의 티커 정보 
        signal

        # 연율화 패러미터
        self.param = param

        # 일별 수익률
        self.rets = price.pct_change().fillna(0)

        # 기대수익률
        self.er = np.array(self.rets * self.param)

        # 변동성
        self.vol = np.array(self.rets.rolling(self.param).std() * np.sqrt(self.param))

        # 공분산행렬
        cov = self.rets.rolling(self.param).cov().dropna() * self.param
        self.cov = cov.values.reshape(int(cov.shape[0]/cov.shape[1]), cov.shape[1], cov.shape[1])

    # EW
    def ew(self, er):
        noa = er.shape[0]
        weights = np.ones_like(er) * (1/noa)
        return weights
    
    # MSR
    def msr(self, er, cov):
        noa = er.shape[0]
        init_guess = np.repeat(1/noa, noa)

        bounds = ((0.0, 1.0), ) * noa
        weights_sum_to_1 = {'type': 'eq',
                            'fun': lambda weights: np.sum(weights) - 1}

        def neg_sharpe(weights, er, cov):
            r = weights.T @ er
            vol = np.sqrt(weights.T @ cov @ weights)
            return - r / vol

        weights = minimize(neg_sharpe,
                        init_guess,
                        args=(er, cov),
                        method='SLSQP',
                        constraints=(weights_sum_to_1,), 
                        bounds=bounds)

        return weights.x
    
    # GMV
    def gmv(self, cov):
        noa = cov.shape[0]
        init_guess = np.repeat(1/noa, noa)

        bounds = ((0.0, 1.0), ) * noa
        weights_sum_to_1 = {'type': 'eq',
                            'fun': lambda weights: np.sum(weights) - 1}

        def port_vol(weights, cov):
            vol = np.sqrt(weights.T @ cov @ weights)
            return vol

        weights = minimize(port_vol, init_guess, args=(cov), method='SLSQP', constraints=(weights_sum_to_1,), bounds=bounds)

        return weights.x
    
    # MDP
    def mdp(self, vol, cov):
        noa = vol.shape[0]
        init_guess = np.repeat(1/noa, noa)
        bounds = ((0.0, 1.0), ) * noa
        
        weights_sum_to_1 = {'type': 'eq',
                            'fun': lambda weights: np.sum(weights) - 1}
        
        def neg_div_ratio(weights, vol, cov):
            weighted_vol = weights.T @ vol
            port_vol = np.sqrt(weights.T @ cov @ weights)
            return - weighted_vol / port_vol
        
        weights = minimize(neg_div_ratio, 
                            init_guess, 
                            args=(vol, cov),
                            method='SLSQP',
                            constraints=(weights_sum_to_1,), 
                            bounds=bounds)
        
        return weights.x
    
    # RP
    def rp(self, cov):
        noa = cov.shape[0]
        init_guess = np.repeat(1/noa, noa)
        bounds = ((0.0, 1.0), ) * noa
        target_risk = np.repeat(1/noa, noa)
        
        weights_sum_to_1 = {'type': 'eq',
                    'fun': lambda weights: np.sum(weights) - 1}
        
        def msd_risk(weights, target_risk, cov):
            
            port_var = weights.T @ cov @ weights
            marginal_contribs = cov @ weights
            
            risk_contribs = np.multiply(marginal_contribs, weights.T) / port_var
            
            w_contribs = risk_contribs
            return ((w_contribs - target_risk)**2).sum()
        
        weights = minimize(msd_risk, 
                            init_guess,
                            args=(target_risk, cov), 
                            method='SLSQP',
                            constraints=(weights_sum_to_1,),
                            bounds=bounds)
        return weights.x
    
    # EMV
    def emv(self, vol):
        inv_vol = 1 / vol
        weights = inv_vol / inv_vol.sum()

        return weights


In [18]:
# # 종적 배분 모델 클래스
# class TimeSeries:
#     '''
#     '''
#     # 초기화 함수
#     def __init__(self, price, param=52):

#         # 연율화 패러미터
#         self.param = param

#         # 일별 수익률
#         self.rets = price.pct_change().dropna()

#         # 기대수익률
#         self.er = np.array(self.rets * self.param)

#         # 변동성
#         self.vol = np.array(self.rets.rolling(self.param).std() * np.sqrt(self.param))

#         # 공분산행렬
#         cov = self.rets.rolling(self.param).cov().dropna() * self.param
#         self.cov = cov.values.reshape(int(cov.shape[0]/cov.shape[1]), cov.shape[1], cov.shape[1])

#         # 거래비용
#         self.cost = 0.0005

#     # VT   
#     def vt(self, port_rets, param, vol_target=0.1):
#         vol = port_rets.rolling(param).std().fillna(0) * np.sqrt(param)
#         weights = (vol_target / vol).replace([np.inf, -np.inf], 0).shift(1).fillna(0)
#         weights[weights > 1] = 1
#         return weights
    
#     # CVT
#     def cvt(self, port_rets, param, delta=0.01, cvar_target=0.05):
#         def calculate_CVaR(rets, delta=0.01):
#             VaR = rets.quantile(delta)    
#             return rets[rets <= VaR].mean()
        
#         rolling_CVaR = -port_rets.rolling(param).apply(calculate_CVaR, args=(delta,)).fillna(0)
#         weights = (cvar_target / rolling_CVaR).replace([np.inf, -np.inf], 0).shift(1).fillna(0)
#         weights[weights > 1] = 1
#         return weights
    
#     # KL
#     def kl(self, port_rets, param):
#         sharpe_ratio = (port_rets.rolling(param).mean() * np.sqrt(param) / port_rets.rolling(param).std())
#         weights = pd.Series(2 * norm.cdf(sharpe_ratio) - 1, index=port_rets.index).fillna(0)
#         weights[weights < 0] = 0
#         weights = weights.shift(1).fillna(0)
#         return weights
    
#     # CPPI
#     def cppi(self, port_rets, m=3, floor=0.7, init_val=1):
#         n_steps = len(port_rets)
#         port_value = init_val
#         floor_value = init_val * floor
#         peak = init_val

#         port_history = pd.Series(dtype=np.float64).reindex_like(port_rets)
#         weight_history = pd.Series(dtype=np.float64).reindex_like(port_rets)
#         floor_history = pd.Series(dtype=np.float64).reindex_like(port_rets)

#         for step in range(n_steps):
#             peak = np.maximum(peak, port_value)
#             floor_value = peak * floor

#             cushion = (port_value - floor_value) / port_value
#             weight = m * cushion

#             risky_alloc = port_value * weight
#             safe_alloc = port_value * (1 - weight)
#             port_value = risky_alloc * (1 + port_rets.iloc[step]) + safe_alloc

#             port_history.iloc[step] = port_value
#             weight_history.iloc[step] = weight
#             floor_history.iloc[step] = floor_value

#         return weight_history.shift(1).fillna(0)


In [20]:
from functools import reduce

def calculate_portvals(price_df: pd.DataFrame, weight_df: pd.DataFrame) -> pd.DataFrame:
    """calculate_portvals

    Args:
        price_df (pd.DataFrame): 
        - DataFrame -> 일별 종가를 담고 있는 df
        weight_df (pd.DataFrame): 
        - DataFrame -> 팩터, 최적화가 끝난 최종 투자비중 df

    Returns:
        pd.DataFrame -> 최종 투자비중에 일별 가격의 변화를 반영했을 때의 투자비중의 변동 => 포트폴리오에 담긴 자산별 가치의 변화를 보여줌 
    """
 
    
    cum_rtn_up_until_now = 1 
    individual_port_val_df_list = []
    prev_end_day = weight_df.index[0]
    
    for end_day in weight_df.index[1:]:
        sub_price_df = price_df.loc[prev_end_day:end_day]
        sub_asset_flow_df = sub_price_df/sub_price_df.iloc[0]

        weight_series = weight_df.loc[prev_end_day]
        indi_port_cum_rtn_series = (sub_asset_flow_df*weight_series)*cum_rtn_up_until_now
    
        individual_port_val_df_list.append(indi_port_cum_rtn_series)

        total_port_cum_rtn_series = indi_port_cum_rtn_series.sum(axis=1)
        cum_rtn_up_until_now = total_port_cum_rtn_series.iloc[-1]

        prev_end_day = end_day 

    individual_port_val_df = reduce(lambda x, y: pd.concat([x, y.iloc[1:]]), individual_port_val_df_list)
    return individual_port_val_df

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce

class BackTest:
    def __init__(self, individual_port_val_df: pd.DataFrame, cost: str) -> pd.DataFrame:
        
        self.portval_df = individual_port_val_df.sum(axis=1)
        
        

    # def transaction_cost(self, weights_df, rets_df, cost=0.0005):


    #     return cost_df


    def get_returns_df(individual_port_val_df: pd.DataFrame, N: int=1, log: bool) -> pd.DataFrame:
        '''
        simple returns or log returns of periodic portfolio value 
        N(# of lookback window) = 1
        '''
        
        df = individual_port_val_df
        
        if log:
            return np.log(df/df.shift(N)).iloc[N-1:].fillna(0)
        
        else:
            return df.pct_change(N, fill_method=None).iloc[N-1:].fillna(0)


    def cum_returns_df(return_df: pd.DataFrame, log: bool) -> pd.DataFrame:
        '''
        cumulated returns 
        '''
        
        if log:
            return np.exp(return_df.cumsum())
        
        else:
            
            # same with (return_df.cumsum() + 1)
            return (1 + return_df).cumprod()   


    def CAGR_series(cum_rtn_df: pd.DataFrame, num_day_in_year: int) -> pd.Series:
        '''
        Compound Annual Growth Rate(CAGR)
        usually, num_day_in_year would be 252
        '''
        
        cagr_series = cum_rtn_df.iloc[-1]**(num_day_in_year/(len(cum_rtn_df))) - 1
        return cagr_series

    def sharpe_ratio(log_rtn_df: pd.DataFrame, num_day_in_year:int, yearly_rfr: int) -> pd.Series:
        '''
        Sharpe Ratio
        yearly_rfr stands for yearly risk free rate
        '''
        
        excess_rtns = log_rtn_df.mean()*num_day_in_year - yearly_rfr
        return excess_rtns / (log_rtn_df.std() * np.sqrt(num_day_in_year))

    def drawdown_infos(cum_returns_df: pd.DataFrame) -> tuple: 
        '''
        drawdown infos: drawdown, maximum drawdown, longest drawdown period
        drawdown: pd.DataFrame
        maximum drawdown: pd.Series
        longest drawdown period: pd.DataFrame
        '''
        
        # 1. Drawdown
        cummax_df = cum_returns_df.cummax()
        dd_df = cum_returns_df/cummax_df - 1
    
        # 2. Maximum drawdown
        mdd_series = dd_df.min()

        # 3. longest_dd_period
        dd_duration_info_list = list()
        max_point_df = dd_df[dd_df == 0]
        
        for col in max_point_df:
            _df = max_point_df[col]
            _df.loc[dd_df[col].last_valid_index()] = 0
            _df = _df.dropna()

            periods = _df.index[1:] - _df.index[:-1]

            days = periods.days
            max_idx = days.argmax()

            longest_dd_period = days.max()
            dd_mean = int(np.mean(days))
            dd_std = int(np.std(days))

            dd_duration_info_list.append(
                [
                    dd_mean,
                    dd_std,
                    longest_dd_period,
                    "{} ~ {}".format(_df.index[:-1][max_idx].date(), _df.index[1:][max_idx].date())
                ]
            )

        dd_duration_info_df = pd.DataFrame(
            dd_duration_info_list,
            index=dd_df.columns,
            columns=['drawdown mean', 'drawdown std', 'longest days', 'longest period']
        )
        return dd_df, mdd_series, dd_duration_info_df
